In [2]:
import numpy as np
import cv2

In [3]:
def load_image(path_img):
    img = cv2.imread(path_img)
    return img

def contours_img(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def filter_shape(contour):
    peri = cv2.arcLength(contour, True)
    
    approx = cv2.approxPolyDP(contour, 0.01 * peri, True)

    if len(approx) <= 7:
        shape = "Building"
        
    else:
        shape = "Tree"
            
    return shape
    
def filter_contours(contours, img_draw):
    count = [0, 0]
    sum_area = [0, 0]
    for x in range(len(contours)):
        shape = filter_shape(contours[x])
        #print(shape)
        
     # if a contour has not contours inside of it, draw the shape filled
        c = hier[0][x][2]
        if c == -1:
            area =  cv2.contourArea(contours[x])
            
            if area > 280 :
                count[1] += 1
                sum_area[1] +=  area
                
                cv2.drawContours(img_draw,[contours[x]],0,(0,0,255),-1)
                
            elif shape == "Tree" :# Tree
                count[0] += 1
                sum_area[0] +=  area
                cv2.drawContours(img_draw,[contours[x]],0,(0,255,0),-1)
                #cv2.drawContours(img_draw,[contours[x]],0,(0,255,0),1)
                
            elif shape == "Building": # Building
                count[1] += 1
                sum_area[1] +=  area
                
                cv2.drawContours(img_draw,[contours[x]],0,(0,0,255),-1)
                #cv2.drawContours(img_draw,[contours[x]],0,(0,255,0),1)
        else:
            cv2.drawContours(img_draw,[contours[x]],0,(0,0,0),-1)
    
    return img_draw, count, sum_area



In [5]:
path_img = './pic/pre-process2.png'
img = load_image(path_img)


#img = cv2.resize(img, None,fx=0.5,fy=0.5)

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)

thresh = cv2.adaptiveThreshold(img_gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,1)

#inv_thresh = cv2.bitwise_not(thresh)

kernel = np.ones((1,1), np.uint8)
#closing  = cv2.morphologyEx(inv_thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)


img_draw = img
img = closing

cv2.imshow("Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
contours, hier = cv2.findContours(img,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

img_draw, count, sum_area = filter_contours(contours, img_draw)

print('Tree', count[0], sum_area[0])
print('Building', count[1], sum_area[1])

cv2.imshow("Result", img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

Tree 500 64070.5
Building 249 144116.5


In [15]:
#img_draw = draw_text_on_image(img_draw, count_yellow, count_orange, sum_area_yellow, sum_area_orange)
cv2.imwrite('./pic/output.png', img_draw)


True